In [7]:
import os
import re
import numpy as np
import time
import random
import pandas as pd
import glob
import shutil
import seaborn as sns
import itertools
import math
from copy import deepcopy
from dataclasses import dataclass
from types import SimpleNamespace

from matplotlib.lines import Line2D
from scipy.integrate import solve_ivp
from sklearn.metrics import confusion_matrix
from autocatalytic_cores_lib import *
import networkx as nx
import matplotlib.pyplot as plt
from scipy.stats import linregress
from sympy import symbols, Matrix, diff, lambdify
from scipy.linalg import eigvals
from scipy.optimize import linprog
from scipy.optimize import minimize
from numpy.linalg import svd
from scipy.sparse import csr_matrix
from scipy.sparse import bmat
from scipy.sparse.csgraph import connected_components
from scipy.linalg import eig
from scipy.optimize import fsolve
from textwrap import dedent

In [8]:
class Neumann(object):

    """
    This class describes the Generalized von Neumann growth model as it was
    discussed in Kemeny et al. (1956, ECTA) and Gale (1960, Chapter 9.5):

    Let:
    n ... number of goods
    m ... number of activities
    A ... input matrix is m-by-n
        a_{i,j} - amount of good j consumed by activity i
    B ... output matrix is m-by-n
        b_{i,j} - amount of good j produced by activity i

    x ... intensity vector (m-vector) with non-negative entries
        x'B - the vector of goods produced
        x'A - the vector of goods consumed
    p ... price vector (n-vector) with non-negative entries
        Bp - the revenue vector for every activity
        Ap - the cost of each activity

    Both A and B have non-negative entries. Moreover, we assume that
    (1) Assumption I (every good which is consumed is also produced):
        for all j, b_{.,j} > 0, i.e. at least one entry is strictly positive
    (2) Assumption II (no free lunch):
        for all i, a_{i,.} > 0, i.e. at least one entry is strictly positive

    Parameters
    ----------
    A : array_like or scalar(float)
        Part of the state transition equation.  It should be `n x n`
    B : array_like or scalar(float)
        Part of the state transition equation.  It should be `n x k`
    """

    def __init__(self, A, B):

        self.A, self.B = list(map(self.convert, (A, B)))
        self.m, self.n = self.A.shape

        # Check if (A, B) satisfy the basic assumptions
        assert self.A.shape == self.B.shape, 'The input and output matrices \
              must have the same dimensions!'
        assert (self.A >= 0).all() and (self.B >= 0).all(), 'The input and \
              output matrices must have only non-negative entries!'

        # (1) Check whether Assumption I is satisfied:
        if (np.sum(B, 0) <= 0).any():
            self.AI = False
        else:
            self.AI = True

        # (2) Check whether Assumption II is satisfied:
        if (np.sum(A, 1) <= 0).any():
            self.AII = False
        else:
            self.AII = True

    def __repr__(self):
        return self.__str__()

    def __str__(self):

        me = """
        Generalized von Neumann expanding model:
          - number of goods          : {n}
          - number of activities     : {m}

        Assumptions:
          - AI:  every column of B has a positive entry    : {AI}
          - AII: every row of A has a positive entry       : {AII}

        """
        # Irreducible                                       : {irr}
        return dedent(me.format(n=self.n, m=self.m,
                                AI=self.AI, AII=self.AII))

    def convert(self, x):
        """
        Convert array_like objects (lists of lists, floats, etc.) into
        well-formed 2D NumPy arrays
        """
        return np.atleast_2d(np.asarray(x))


    def bounds(self):
        """
        Calculate the trivial upper and lower bounds for alpha (expansion rate)
        and beta (interest factor). See the proof of Theorem 9.8 in Gale (1960)
        """

        n, m = self.n, self.m
        A, B = self.A, self.B

        f = lambda α: ((B - α * A) @ np.ones((n, 1))).max()
        g = lambda β: (np.ones((1, m)) @ (B - β * A)).min()

        UB = fsolve(f, 1).item()  # Upper bound for α, β
        LB = fsolve(g, 2).item()  # Lower bound for α, β

        return LB, UB


    def zerosum(self, γ, dual=False):
        """
        Given gamma, calculate the value and optimal strategies of a
        two-player zero-sum game given by the matrix

                M(gamma) = B - gamma * A

        Row player maximizing, column player minimizing

        Zero-sum game as an LP (primal --> α)

            max (0', 1) @ (x', v)
            subject to
            [-M', ones(n, 1)] @ (x', v)' <= 0
            (x', v) @ (ones(m, 1), 0) = 1
            (x', v) >= (0', -inf)

        Zero-sum game as an LP (dual --> beta)

            min (0', 1) @ (p', u)
            subject to
            [M, -ones(m, 1)] @ (p', u)' <= 0
            (p', u) @ (ones(n, 1), 0) = 1
            (p', u) >= (0', -inf)

        Outputs:
        --------
        value: scalar
            value of the zero-sum game

        strategy: vector
            if dual = False, it is the intensity vector,
            if dual = True, it is the price vector
        """

        A, B, n, m = self.A, self.B, self.n, self.m
        M = B - γ * A

        if dual == False:
            # Solve the primal LP (for details see the description)
            # (1) Define the problem for v as a maximization (linprog minimizes)
            c = np.hstack([np.zeros(m), -1])

            # (2) Add constraints :
            # ... non-negativity constraints
            bounds = tuple(m * [(0, None)] + [(None, None)])
            # ... inequality constraints
            A_iq = np.hstack([-M.T, np.ones((n, 1))])
            b_iq = np.zeros((n, 1))
            # ... normalization
            A_eq = np.hstack([np.ones(m), 0]).reshape(1, m + 1)
            b_eq = 1

            res = linprog(c, A_ub=A_iq, b_ub=b_iq, A_eq=A_eq, b_eq=b_eq,
                          bounds=bounds)

        else:
            # Solve the dual LP (for details see the description)
            # (1) Define the problem for v as a maximization (linprog minimizes)
            c = np.hstack([np.zeros(n), 1])

            # (2) Add constraints :
            # ... non-negativity constraints
            bounds = tuple(n * [(0, None)] + [(None, None)])
            # ... inequality constraints
            A_iq = np.hstack([M, -np.ones((m, 1))])
            b_iq = np.zeros((m, 1))
            # ... normalization
            A_eq = np.hstack([np.ones(n), 0]).reshape(1, n + 1)
            b_eq = 1

            res = linprog(c, A_ub=A_iq, b_ub=b_iq, A_eq=A_eq, b_eq=b_eq,
                          bounds=bounds)

        if res.status != 0 or res.x is None:
            # LP infeasible or error
            return np.nan, None

        # Pull out the required quantities
        value = res.x[-1]
        strategy = res.x[:-1]

        return value, strategy


    def expansion(self, tol=1e-8, maxit=1000):
        """
        The algorithm used here is described in Hamburger-Thompson-Weil
        (1967, ECTA). It is based on a simple bisection argument and utilizes
        the idea that for a given γ (= α or β), the matrix "M = B - γ * A"
        defines a two-player zero-sum game, where the optimal strategies are
        the (normalized) intensity and price vector.

        Outputs:
        --------
        alpha: scalar
            optimal expansion rate
        """

        LB, UB = self.bounds()

        for iter in range(maxit):

            γ = (LB + UB) / 2
            ZS = self.zerosum(γ=γ, dual=False)
            V = ZS[0]     # value of the game with γ

            if V >= 0:
                LB = γ
            else:
                UB = γ

            if abs(UB - LB) < tol:
                γ = (UB + LB) / 2
                x = self.zerosum(γ=γ)[1]
                p = self.zerosum(γ=γ, dual=True)[1]
                break

        return γ, x, p

    def interest(self, tol=1e-8, maxit=1000):
        """
        The algorithm used here is described in Hamburger-Thompson-Weil
        (1967, ECTA). It is based on a simple bisection argument and utilizes
        the idea that for a given gamma (= alpha or beta),
        the matrix "M = B - γ * A" defines a two-player zero-sum game,
        where the optimal strategies are the (normalized) intensity and price
        vector

        Outputs:
        --------
        beta: scalar
            optimal interest rate
        """

        LB, UB = self.bounds()

        for iter in range(maxit):
            γ = (LB + UB) / 2
            ZS = self.zerosum(γ=γ, dual=True)
            V = ZS[0]

            if V > 0:
                LB = γ
            else:
                UB = γ

            if abs(UB - LB) < tol:
                γ = (UB + LB) / 2
                p = self.zerosum(γ=γ, dual=True)[1]
                x = self.zerosum(γ=γ)[1]
                break

        return γ, x, p
    
def compute_von_neumann_alpha_beta(S_plus, S_minus, tol=1e-8):
    """
    Compute von Neumann alpha (expansion), beta (interest),
    and the optimal normalized flows for both problems.

    Returns:
    --------
    alpha : Optimal expansion rate.
    beta : Optimal interest rate.
    x_alpha : Optimal intensity vector (normalized flow) for expansion.
    p_alpha : Optimal price vector for expansion.
    x_beta : Optimal intensity vector (normalized flow) for interest.
    p_beta : Optimal price vector for interest.
    """
    
    A = S_minus.T
    B = S_plus.T
    model = Neumann(A, B)

    # alpha
    alpha, x_alpha, p_alpha = model.expansion(tol=tol)

    # beta
    beta, x_beta, p_beta  = model.interest(tol=tol)

    return alpha, beta, x_alpha, p_alpha, x_beta, p_beta

In [3]:
def fmt(z, prec=4):
    """Format complex numbers"""
    r, i = z.real, z.imag
    if abs(r) < 10**(-prec) and abs(i) < 10**(-prec):
        return "0"
    if abs(i) < 10**(-prec):
        return f"{r:.{prec}f}"
    if abs(r) < 10**(-prec):
        return f"{i:.{prec}f}j"
    sign = "+" if i>=0 else "-"
    return f"{r:.{prec}f}{sign}{abs(i):.{prec}f}j"

In [4]:
'''
Compute Core eigenvetor
As cores are autonomus and reversible, 
we can also seperate the core into reversible S+ and S-
So the problem is actually simple eigen value problem
lambda*x = [S+]*[S-]^(-1)*x
'''

'\nCompute Core eigenvetor\nAs cores are autonomus and reversible, \nwe can also seperate the core into reversible S+ and S-\nSo the problem is actually simple eigen value problem\nlambda*x = [S+]*[S-]^(-1)*x\n'

In [6]:
# compute the topological growth bound
def compute_topological_growth_bound(S_minus, alpha, x_alpha):
    x_alpha = x_alpha / np.sum(x_alpha)  # Normalize to make max(x_alpha) = 1
    sp = np.dot(S_minus, x_alpha)
    mu = (alpha - 1) * sp.sum()
    return mu

def compute_Sx(S_plus, S_minus, x_alpha):
    x_alpha = x_alpha / np.sum(x_alpha)
    Stot = S_plus - S_minus
    sp = np.dot(Stot, x_alpha)
    return sp

In [11]:
def generalized_eig_core(Q):
    """
    Solve the generalized eigenvalue problem S_plus x = λ S_minus x for a reversible, autonomous core Q,
    where Q = S_plus - S_minus. Returns eigenvalues sorted by real part (descending) and corresponding eigenvectors.
    """
    # Decompose Q into S_plus (off-diagonals) and S_minus (diagonal)
    S_plus = np.maximum(Q, 0)
    S_minus = np.maximum(-Q, 0)

    # Solve generalized eigenvalue problem
    eigenvals, eigenvecs = eig(S_plus, S_minus)
    alpha, beta, x_alpha, p_alpha, x_beta, p_beta = compute_von_neumann_alpha_beta(S_plus, S_minus)

    # Sort by real part of eigenvalues, descending
    idx = np.argsort(-eigenvals.real)

    # Compute Growth Bound
    mu = compute_topological_growth_bound(S_minus, alpha, x_alpha)
    
    return eigenvals[idx], eigenvecs[:, idx], alpha, beta, x_alpha, mu

In [14]:
'''
Motif 1 and 2
'''
Q = np.array([[-1, 0, 2, 0, 0],
              [1, -1, 0, 0, 0],
              [0, 1, -1, 0, 0],
              [0, 0, 0, 1, -2],
              [0, 0, 0, -1, 1]])

lambdas, vectors, alpha, beta, x_alpha, mu = generalized_eig_core(Q)

print(f"alpha = {alpha:.4f}, beta = {beta:.4f}, optimal_flow = {x_alpha}, mu = {mu:.4f}\n")
for mode, (lam, vec) in enumerate(zip(lambdas, vectors.T), start=1):
    print(f"eigenvalue {mode}: λ = {fmt(lam)}")
    vs = [fmt(x) for x in vec]
    print("  v = [" + ", ".join(vs) + "]\n")

alpha = 1.2599, beta = 0.7071, optimal_flow = [4.12598948e-01 3.27480002e-01 2.59921050e-01 0.00000000e+00
 3.31744410e-10], mu = 0.2599

eigenvalue 1: λ = 1.2599
  v = [0.7024, 0.5575, 0.4425, 0, 0]

eigenvalue 2: λ = 0.7071
  v = [0, 0, 0, -0.8165, -0.5774]

eigenvalue 3: λ = -0.6300+1.0911j
  v = [0.5120+0.4809j, 0.1273-0.5428j, -0.4236+0.1279j, 0, 0]

eigenvalue 4: λ = -0.6300-1.0911j
  v = [0.5120-0.4809j, 0.1273+0.5428j, -0.4236-0.1279j, 0, 0]

eigenvalue 5: λ = -0.7071
  v = [0, 0, 0, -0.8165, 0.5774]



In [32]:
'''
Motif 1
'''
Q1 = np.array([[1, -1],
              [-1, 2]])

lambdas, vectors, alpha, beta, x_alpha, mu = generalized_eig_core(Q1)

print(f"alpha = {alpha:.4f}, beta = {beta:.4f}, optimal_flow = {x_alpha}, mu = {mu:.4f}\n")
for mode, (lam, vec) in enumerate(zip(lambdas, vectors.T), start=1):
    print(f"eigenvalue {mode}: λ = {fmt(lam)}")
    vs = [fmt(x) for x in vec]
    print("  v = [" + ", ".join(vs) + "]\n")

alpha = 1.4142, beta = 1.4142, optimal_flow = [0.58578644 0.41421356], mu = 0.4142

eigenvalue 1: λ = 1.4142
  v = [-0.8165, -0.5774]

eigenvalue 2: λ = -1.4142
  v = [-0.8165, 0.5774]



In [30]:
'''
Motif 2
'''
Q2 = np.array([[-1, 0, 2],
              [1, -1, 0],
              [0, 1, -1]])

lambdas, vectors, alpha, beta, x_alpha, mu = generalized_eig_core(Q2)

print(f"alpha = {alpha:.4f}, beta = {beta:.4f}, optimal_flow = {x_alpha}, mu = {mu:.4f}\n")
for mode, (lam, vec) in enumerate(zip(lambdas, vectors.T), start=1):
    print(f"eigenvalue {mode}: λ = {fmt(lam)}")
    vs = [fmt(x) for x in vec]
    print("  v = [" + ", ".join(vs) + "]\n")

alpha = 1.2599, beta = 1.2599, optimal_flow = [0.41259895 0.32748    0.25992105], mu = 0.2599

eigenvalue 1: λ = 1.2599
  v = [0.7024, 0.5575, 0.4425]

eigenvalue 2: λ = -0.6300+1.0911j
  v = [0.5120+0.4809j, 0.1273-0.5428j, -0.4236+0.1279j]

eigenvalue 3: λ = -0.6300-1.0911j
  v = [0.5120-0.4809j, 0.1273+0.5428j, -0.4236-0.1279j]



In [ ]:
'''
Motif 3, 4, 5
[-1, c, e]
[a, -1, f]
[b, d, -1]
'''

In [16]:
'''
Motif 3
'''
Q3 = np.array([[-1, 1, 1],
               [1, -1, 0],
               [1, 0, -1]])

lambdas, vectors, alpha, beta, x_alpha, mu = generalized_eig_core(Q3)

print(f"alpha = {alpha:.4f}, beta = {beta:.4f}, optimal_flow = {x_alpha}, mu = {mu:.4f}\n")
for mode, (lam, vec) in enumerate(zip(lambdas, vectors.T), start=1):
    print(f"eigenvalue {mode}: λ = {fmt(lam)}")
    vs = [fmt(x) for x in vec]
    print("  v = [" + ", ".join(vs) + "]\n")

alpha = 1.4142, beta = 1.4142, optimal_flow = [0.41421356 0.29289322 0.29289322], mu = 0.4142

eigenvalue 1: λ = 1.4142
  v = [-0.7071, -0.5000, -0.5000]

eigenvalue 2: λ = 0
  v = [0, -0.7071, 0.7071]

eigenvalue 3: λ = -1.4142
  v = [-0.7071, 0.5000, 0.5000]



In [15]:
'''
Motif 4
'''
Q4 = np.array([[-1, 1, 1],
               [1, -1, 0],
               [1, 1, -1]])

lambdas, vectors, alpha, beta, x_alpha, mu = generalized_eig_core(Q4)

print(f"alpha = {alpha:.4f}, beta = {beta:.4f}, optimal_flow = {x_alpha}, mu = {mu:.4f}\n")
for mode, (lam, vec) in enumerate(zip(lambdas, vectors.T), start=1):
    print(f"eigenvalue {mode}: λ = {fmt(lam)}")
    vs = [fmt(x) for x in vec]
    print("  v = [" + ", ".join(vs) + "]\n")

alpha = 1.6180, beta = 1.6180, optimal_flow = [0.38196601 0.23606798 0.38196601], mu = 0.6180

eigenvalue 1: λ = 1.6180
  v = [-0.6479, -0.4004, -0.6479]

eigenvalue 2: λ = -0.6180
  v = [0.4653, -0.7529, 0.4653]

eigenvalue 3: λ = -1.0000
  v = [-0.7071, 0.7071, 0]



In [17]:
'''
Motif 5
'''
Q5 = np.array([[-1, 1, 1],
               [1, -1, 1],
               [1, 1, -1]])

lambdas, vectors, alpha, beta, x_alpha, mu = generalized_eig_core(Q5)

print(f"alpha = {alpha:.4f}, beta = {beta:.4f}, optimal_flow = {x_alpha}, mu = {mu:.4f}\n")
for mode, (lam, vec) in enumerate(zip(lambdas, vectors.T), start=1):
    print(f"eigenvalue {mode}: λ = {fmt(lam)}")
    vs = [fmt(x) for x in vec]
    print("  v = [" + ", ".join(vs) + "]\n")

alpha = 2.0000, beta = 2.0000, optimal_flow = [0.33333333 0.33333333 0.33333333], mu = 1.0000

eigenvalue 1: λ = 2.0000
  v = [0.5774, 0.5774, 0.5774]

eigenvalue 2: λ = -1.0000
  v = [0.8165, -0.4082, -0.4082]

eigenvalue 3: λ = -1.0000
  v = [0.0215, -0.7176, 0.6961]



In [21]:
'''
Motif 5 order 3 not full
'''
Q5 = np.array([[-1, 1, 1],
               [0.5, -1, 2.5],
               [1, 1, -1]])

lambdas, vectors, alpha, beta, x_alpha, mu = generalized_eig_core(Q5)

print(f"alpha = {alpha:.4f}, beta = {beta:.4f}, optimal_flow = {x_alpha}, mu = {mu:.4f}\n")
for mode, (lam, vec) in enumerate(zip(lambdas, vectors.T), start=1):
    print(f"eigenvalue {mode}: λ = {fmt(lam)}")
    vs = [fmt(x) for x in vec]
    print("  v = [" + ", ".join(vs) + "]\n")

alpha = 2.3028, beta = 2.3028, optimal_flow = [0.30277564 0.39444872 0.30277564], mu = 1.3028

eigenvalue 1: λ = 2.3028
  v = [0.5201, 0.6775, 0.5201]

eigenvalue 2: λ = -1.0000
  v = [-0.5883, 0.7845, -0.1961]

eigenvalue 3: λ = -1.3028
  v = [0.3700, -0.8521, 0.3700]



In [22]:
'''
Motif 5 order 3 not full
'''
Q5 = np.array([[-1, 1, 1],
               [1, -1, 2],
               [1, 1, -1]])

lambdas, vectors, alpha, beta, x_alpha, mu = generalized_eig_core(Q5)

print(f"alpha = {alpha:.4f}, beta = {beta:.4f}, optimal_flow = {x_alpha}, mu = {mu:.4f}\n")
for mode, (lam, vec) in enumerate(zip(lambdas, vectors.T), start=1):
    print(f"eigenvalue {mode}: λ = {fmt(lam)}")
    vs = [fmt(x) for x in vec]
    print("  v = [" + ", ".join(vs) + "]\n")

alpha = 2.3028, beta = 2.3028, optimal_flow = [0.30277564 0.39444872 0.30277564], mu = 1.3028

eigenvalue 1: λ = 2.3028
  v = [0.5201, 0.6775, 0.5201]

eigenvalue 2: λ = -1.0000
  v = [-0.7071, 0.7071, 0]

eigenvalue 3: λ = -1.3028
  v = [0.3700, -0.8521, 0.3700]



In [27]:
'''
Motif 5 order 3
'''
Q5 = np.array([[-1, 1.5, 1.5],
               [0.1, -1, 2.9],
               [2, 1, -1]])

lambdas, vectors, alpha, beta, x_alpha, mu = generalized_eig_core(Q5)

print(f"alpha = {alpha:.4f}, beta = {beta:.4f}, optimal_flow = {x_alpha}, mu = {mu:.4f}\n")
for mode, (lam, vec) in enumerate(zip(lambdas, vectors.T), start=1):
    print(f"eigenvalue {mode}: λ = {fmt(lam)}")
    vs = [fmt(x) for x in vec]
    print("  v = [" + ", ".join(vs) + "]\n")

alpha = 3.0000, beta = 3.0000, optimal_flow = [0.33333333 0.33333333 0.33333333], mu = 2.0000

eigenvalue 1: λ = 3.0000
  v = [0.5774, 0.5774, 0.5774]

eigenvalue 2: λ = -1.5000+0.8367j
  v = [0.0840+0.3891j, -0.7550-0.2300j, 0.4540-0.1123j]

eigenvalue 3: λ = -1.5000-0.8367j
  v = [0.0840-0.3891j, -0.7550+0.2300j, 0.4540+0.1123j]



In [26]:
'''
Motif 5 order 3
'''
Q5 = np.array([[-1, 0.1, 2.9],
               [0.1, -1, 2.9],
               [0.5, 2.5, -1]])

lambdas, vectors, alpha, beta, x_alpha, mu = generalized_eig_core(Q5)

print(f"alpha = {alpha:.4f}, beta = {beta:.4f}, optimal_flow = {x_alpha}, mu = {mu:.4f}\n")
for mode, (lam, vec) in enumerate(zip(lambdas, vectors.T), start=1):
    print(f"eigenvalue {mode}: λ = {fmt(lam)}")
    vs = [fmt(x) for x in vec]
    print("  v = [" + ", ".join(vs) + "]\n")

alpha = 3.0000, beta = 3.0000, optimal_flow = [0.33333333 0.33333333 0.33333333], mu = 2.0000

eigenvalue 1: λ = 3.0000
  v = [0.5774, 0.5774, 0.5774]

eigenvalue 2: λ = -0.1000
  v = [-0.9804, 0.1950, 0.0271]

eigenvalue 3: λ = -2.9000
  v = [-0.5707, -0.5707, 0.5904]



In [28]:
'''
Motif 5 order 3
'''
Q5 = np.array([[-1, 1, 2],
               [1, -1, 2],
               [1, 2, -1]])

lambdas, vectors, alpha, beta, x_alpha, mu = generalized_eig_core(Q5)

print(f"alpha = {alpha:.4f}, beta = {beta:.4f}, optimal_flow = {x_alpha}, mu = {mu:.4f}\n")
for mode, (lam, vec) in enumerate(zip(lambdas, vectors.T), start=1):
    print(f"eigenvalue {mode}: λ = {fmt(lam)}")
    vs = [fmt(x) for x in vec]
    print("  v = [" + ", ".join(vs) + "]\n")

alpha = 3.0000, beta = 3.0000, optimal_flow = [0.33333333 0.33333333 0.33333333], mu = 2.0000

eigenvalue 1: λ = 3.0000
  v = [0.5774, 0.5774, 0.5774]

eigenvalue 2: λ = -1.0000
  v = [0.9045, -0.3015, -0.3015]

eigenvalue 3: λ = -2.0000
  v = [-0.4851, -0.4851, 0.7276]



In [6]:
'''
Motif 5 order 4
'''
Q5 = np.array([[-1, 2, 2],
               [2, -1, 2],
               [2, 2, -1]])

lambdas, vectors, alpha, beta = generalized_eig_core(Q5)

print(f"alpha = {alpha:.4f}, beta = {beta:.4f}\n")
for mode, (lam, vec) in enumerate(zip(lambdas, vectors.T), start=1):
    print(f"eigenvalue {mode}: λ = {fmt(lam)}")
    vs = [fmt(x) for x in vec]
    print("  v = [" + ", ".join(vs) + "]\n")

alpha = 4.0000, beta = 4.0000

eigenvalue 1: λ = 4.0000
  v = [0.5774, 0.5774, 0.5774]

eigenvalue 2: λ = -2.0000
  v = [0.8165, -0.4082, -0.4082]

eigenvalue 3: λ = -2.0000
  v = [0.0215, -0.7176, 0.6961]

